In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
# {k:np.sum if k in {'col3','col4'} else k:np.average for k in df.columns if k not in {'col1','col2'}

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
acc_uid = 'account_uid'
application_id = 'application_id'

In [5]:
datetime_cols = [
'reporting_dt',
'fund_date', 
'trade_opened_dt',
'trade_close_dt', 
'paymnt_condition_principal_terms_amt_dt',
'paymnt_condition_interest_terms_amt_dt',       
'paymnt_condition_grace_start_dt', 
'paymnt_condition_grace_end_dt',
'paymnt_condition_interest_payment_due_date',       
# 'overall_val_credit_total_amt_date', # Дата расчета полной стоимости кредита (займа) - возможно от данного столбца можно избавиться, т.к. сама по себе дата расчета показателя ни на что не должна влиять
# 'month_aver_paymt_calc_date', # тоже можно избавиться по тем же причинам
'collat_insured_insur_start_dt',
'collat_insured_insur_end_dt', 
'collat_insured_insur_fact_end_dt',       
'collat_repay_dt', 
'loan_indicator_dt', 
'legal_items_court_act_dt', 
'hold_dt', 
'file_since_dt', 
'last_updated_dt',
'last_uploaded_dt', 
'arrear_calc_date',
'due_arrear_start_dt',
# 'due_arrear_calc_date', # тоже можно избавиться по тем же причинам
'past_due_dt', 
'past_due_calc_date', 
'past_due_principal_missed_date',
'past_due_int_missed_date']

In [6]:
datetime_cols.append(acc_uid)


In [7]:
cols_continuous_variables = [
'account_amt_credit_limit',
'account_amt_ensured_amt',
'coborrower_solidary_num',
'paymnt_condition_principal_terms_amt',
'paymnt_condition_interest_terms_amt',
'paymnt_condition_min_paymt',
'overall_val_credit_total_amt',
'overall_val_credit_total_monetary_amt',
'month_aver_paymt_aver_paymt_amt',
'collat_insured_insur_limit',
'collat_repay_amt',
'arrear_start_amt_outstanding',
'arrear_amt_outstanding',
'arrear_principal_outstanding',
'arrear_int_outstanding',
'arrear_other_amt_outstanding',
'due_arrear_amt_outstanding',
'due_arrear_principal_outstanding',
'due_arrear_int_outstanding',
'due_arrear_other_amtoutstanding',
'past_due_amt_past_due',
'past_due_principal_amt_past_due',
'past_due_int_amt_past_due',
'past_due_other_amt_past_due',
'delay5',
'delay30',
'delay60',
'delay90',
'delay_more',
'cred_max_overdue',
]

In [8]:
cols_continuous_variables.append(acc_uid)

In [9]:
cols_categorical_variables = [
'trade_owner_indic',
'trade_trade_type_code',
'trade_loan_kind_code',
'trade_acct_type1',
'trade_has_card',
'trade_is_novation',
'trade_is_money_source',
'coborrower_has_solidary',
'coborrower_solidary_num',	
'paymnt_condition_terms_frequency',
'has_collaterals',
'has_guarantees',
'has_indie_guarantees',
'collat_insured_insur_sign',
'collat_insured_has_franchise',
'collat_insured_insur_end_reason',
'colat_repaid',
'collat_repay_code',
'loan_indicator',
'legal_items_has_legal_dispute',
'legal_items_court_act_effect_code',
'hold_code',
'arrear_sign',
'arrear_last_payment_due_code',
'arrear_unconfirm_grace',
'due_arrear_last_payment_due_code',
'past_due_last_payment_due_code',
'attr_value'
]

In [10]:
cols_categorical_variables.append(acc_uid)

In [11]:
cols_to_drop = [
'client_id',
'equifax_id',
'collat_insured_currency_code',
'account_amt_currency_code',
'overall_val_credit_total_amt_date',
'month_aver_paymt_calc_date',
'due_arrear_calc_date'
]

In [12]:
# dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')
df_bki = pd.read_csv('data/df_BKI_30k.csv', index_col=False, low_memory=False)

In [13]:
usd_loan_idx = df_bki.loc[(df_bki['account_amt_currency_code'] == 'USD') | (df_bki['account_amt_currency_code'] == 'USD')].index

In [14]:
df_bki.drop(usd_loan_idx, inplace=True)

In [15]:
df_target = pd.read_csv('data/df_target_30k.csv', index_col=False, low_memory=False)

In [16]:
df_bki.drop(columns=cols_to_drop, inplace=True, index=1)

In [17]:
df_bki.head()

,application_id,reporting_dt,account_uid,fund_date,trade_owner_indic,trade_opened_dt,trade_trade_type_code,trade_loan_kind_code,trade_acct_type1,trade_is_consumer_loan,trade_has_card,trade_is_novation,trade_is_money_source,trade_close_dt,account_amt_credit_limit,account_amt_ensured_amt,coborrower_has_solidary,coborrower_solidary_num,paymnt_condition_principal_terms_amt,paymnt_condition_principal_terms_amt_dt,paymnt_condition_interest_terms_amt,paymnt_condition_interest_terms_amt_dt,paymnt_condition_terms_frequency,paymnt_condition_min_paymt,paymnt_condition_grace_start_dt,paymnt_condition_grace_end_dt,paymnt_condition_interest_payment_due_date,overall_val_credit_total_amt,overall_val_credit_total_monetary_amt,month_aver_paymt_aver_paymt_amt,has_collaterals,has_guarantees,has_indie_guarantees,collat_insured_insur_sign,collat_insured_insur_limit,collat_insured_has_franchise,collat_insured_insur_start_dt,collat_insured_insur_end_dt,collat_insured_insur_fact_end_dt,collat_insured_insur_end_reason,colat_repaid,collat_repay_code,collat_repay_dt,collat_repay_amt,loan_indicator,loan_indicator_dt,legal_items_has_legal_dispute,legal_items_court_act_dt,legal_items_court_act_effect_code,hold_code,hold_dt,file_since_dt,last_updated_dt,last_uploaded_dt,arrear_sign,arrear_start_amt_outstanding,arrear_last_payment_due_code,arrear_amt_outstanding,arrear_principal_outstanding,arrear_int_outstanding,arrear_other_amt_outstanding,arrear_calc_date,arrear_unconfirm_grace,due_arrear_start_dt,due_arrear_last_payment_due_code,due_arrear_amt_outstanding,due_arrear_principal_outstanding,due_arrear_int_outstanding,due_arrear_other_amtoutstanding,past_due_dt,past_due_last_payment_due_code,past_due_amt_past_due,past_due_principal_amt_past_due,past_due_int_amt_past_due,past_due_other_amt_past_due,past_due_calc_date,past_due_principal_missed_date,past_due_int_missed_date,delay5,delay30,delay60,delay90,delay_more,cred_max_overdue,attr_value
0,51859026,2023-08-01,902086582865039990,2014-07-11,1.0,2014-07-11,1.0,1.0,15.0,1.0,0.0,0.0,0.0,2017-07-11,240000.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.060,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0,2017-09-20,NaN,NaN,NaN,NaN,NaN,2014-07-11,2017-09-20,2017-10-01,0.0,240000.00,1.0,0.00,0.00,0.00,0.00,2017-09-20,NaN,NaN,1.0,0.00,0.00,0.00,0.00,NaN,1.0,0.0,0.0,0.0,0.0,2017-09-20,NaN,NaN,2,2,0,1,16,113333.27,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...
2,51859359,2023-08-01,741680445545450197,2012-03-21,1.0,2012-03-21,1.0,5.0,19.0,1.0,0.0,0.0,1.0,2100-01-01,90000.0,NaN,NaN,NaN,845.48,2023-07-31,0.00,NaN,3.0,NaN,NaN,NaN,NaN,0.100,101457.3,845.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-15,2023-06-30,2023-07-07,1.0,90000.00,0.0,101457.30,101457.30,0.00,0.00,2023-06-30,0.0,NaN,0.0,0.00,NaN,NaN,NaN,2017-02-20,0.0,101457.3,101457.3,0.0,0.0,2023-06-30,2017-02-19,NaN,0,0,0,0,1,101457.30,--543100000AAAAAAAA98765432-------------------...
3,51859863,2023-08-01,634072954671362970,2023-05-10,1.0,2023-05-10,1.0,7.0,99.0,1.0,1.0,0.0,1.0,2033-05-10,60000.0,NaN,0.0,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.0,3119.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-10,2023-07-27,2023-07-29,1.0,59599.00,NaN,59551.14,59551.14,0.00,0.00,2023-07-27,0.0,2023-05-10,NaN,59551.14,59551.14,0.00,0.00,NaN,NaN,0.0,NaN,NaN,NaN,2023-07-27,NaN,NaN,0,0,0,0,0,0.00,-000
4,51860912,2023-08-01,446357239045684673,2022-12-11,1.0,2022-12-11,1.0,5.0,99.0,1.0,1.0,0.0,1.0,2260-12-31,39000.0,NaN,0.0,NaN,1198.55,2023-08-11,1801.45,2023-08-11,3.0,NaN,NaN,NaN,NaN,34.443,656488.0,3000.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-11,2023-07-18,2023-07-21,1.0,7822.94,1.0,40167.87,32789.84,1769.89,5608.14,2023-07-16,0.0,2023-01-16,1.0,40167.87,32789.84,1769.89,5608.14,NaN,1.0,0.0,NaN,NaN,NaN,2023-07-16,NaN,NaN,0,2,0,0,0,1195.90,-22020000
5,51861227,2023-08-01,219254709201369890,2017-01-25

In [18]:
df_dates = df_bki[datetime_cols]
df_continuous = df_bki[cols_continuous_variables]
df_categorical = df_bki[cols_categorical_variables]
df_application_id = df_bki[[acc_uid, application_id]]

In [19]:
del df_bki

In [ ]:
# df_bki[(df_bki['application_id'] == 51871261)]['reporting_dt'].value_counts()

reporting_dt
2023-08-01    18
Name: count, dtype: int64

In [11]:
df_target['target'].value_counts(dropna=False)

target
0    23605
1     6395
Name: count, dtype: int64

In [12]:
# df_bki['client_id'].value_counts()

In [13]:
# df_bki['application_id'].value_counts()

In [ ]:
df_test = pd.get_dummies(df_bki.loc[(df_bki['application_id'] == 53123153) | (df_bki['application_id'] == 52175920)][['application_id', 'trade_acct_type1', 'trade_loan_kind_code']], columns=['trade_acct_type1', 'trade_loan_kind_code'], dtype=int)

In [ ]:
# df_test

In [16]:
df_test.groupby(['application_id']).sum()

,trade_acct_type1_1.0,trade_acct_type1_4.7,trade_acct_type1_8.0,trade_acct_type1_9.0,trade_acct_type1_14.0,trade_acct_type1_15.0,trade_acct_type1_19.0,trade_acct_type1_99.0,trade_loan_kind_code_1.0,trade_loan_kind_code_2.0,trade_loan_kind_code_3.0,trade_loan_kind_code_4.0,trade_loan_kind_code_5.0,trade_loan_kind_code_7.0,trade_loan_kind_code_99.0
application_id,,,,,,,,,,,,,,,
52175920,2,1,6,1,1,2,311,59,3,1,366,2,7,3,1
53123153,1,0,0,0,1,2,476,12,2,0,485,0,5,0,0


In [17]:
# df_bki['trade_opened_dt'].value_counts().sort_index(ascending=True)

In [18]:
df_bki['reporting_dt'].value_counts().sort_index(ascending=True)

reporting_dt
2023-07-19      959
2023-07-20     5360
2023-07-21     5230
2023-07-22     5309
2023-07-23     5747
2023-07-24     6510
2023-07-25     5019
2023-07-26     4607
2023-07-28     5420
2023-07-29     6254
2023-07-30     6039
2023-07-31     7319
2023-08-01     6102
2023-08-02     5139
2023-08-03     5860
2023-08-04     5151
2023-08-05     4793
2023-08-06     6231
2023-08-07     6924
2023-08-08     5785
2023-08-09     5996
2023-08-10     5671
2023-08-11     5203
2023-08-12     6768
2023-08-13     7343
2023-08-14     5993
2023-08-15     6748
2023-08-16     6627
2023-08-17     8356
2023-08-18    10381
2023-08-19    10043
2023-08-20    11882
2023-08-21     8321
2023-08-22     8267
2023-08-23    13974
2023-08-24    10790
2023-08-25    10870
2023-08-26    10682
2023-08-27    12030
2023-08-28     9679
2023-08-29    11101
2023-08-30    12693
2023-08-31    11260
2023-09-01    10007
2023-09-02     9573
2023-09-03     7918
2023-09-04    10764
2023-09-05    10460
2023-09-06     9479
2023-09

In [19]:
# df_bki[(df_bki['application_id'] == 53123153)]['trade_opened_dt'].value_counts().sort_index(ascending=True)

In [20]:
# df_bki[(df_bki['application_id'] == 53123153)]['trade_close_dt'].value_counts().sort_index(ascending=True)

In [22]:
df_bki[(df_bki['application_id'] == 53123153)][['trade_opened_dt', 'trade_close_dt']].sort_values('trade_opened_dt').head()

,trade_opened_dt,trade_close_dt
282915,2013-04-11,2018-08-08
246354,2013-12-25,2018-08-06
257865,2014-05-06,2018-05-06
253527,2014-07-04,NaN
253487,2018-05-15,2018-06-04


In [23]:
df_test1 = pd.pivot_table(df_bki[['application_id', 'account_amt_credit_limit', 'trade_loan_kind_code']], values=['account_amt_credit_limit'], index=['application_id'], columns=['trade_loan_kind_code'], aggfunc='sum')

In [24]:
df_test1 = df_test1.fillna(0.0)

In [25]:
df_test1.head()

account_amt_credit_limit                           \
trade_loan_kind_code                     1.0  2.0        3.0      4.0    
application_id                                                           
51534101                            387063.66  0.0  110394.95      0.0   
51534210                              5300.00  0.0   88329.00      0.0   
51534377                                 0.00  0.0   17000.00      0.0   
51534467                            119726.58  0.0       0.00  15000.0   
51534573                            497962.00  0.0  211845.38  16856.0   

                                                    
trade_loan_kind_code      5.0  6.0       7.0  99.0  
application_id                                      
51534101                   0.0  0.0       0.0  0.0  
51534210                   0.0  0.0       0.0  0.0  
51534377               56000.0  0.0       0.0  0.0  
51534467               15000.0  0.0       0.0  0.0  
51534573              188000.0  0.0  190000.0  0.0

In [26]:
df_test1.loc[53123153]

                          trade_loan_kind_code
account_amt_credit_limit  1.0                      799000.00
                          2.0                           0.00
                          3.0                     7899652.09
                          4.0                           0.00
                          5.0                      270000.10
                          6.0                           0.00
                          7.0                           0.00
                          99.0                          0.00
Name: 53123153, dtype: float64

In [27]:
df_test2 = pd.pivot_table(df_bki[['application_id', 'account_amt_credit_limit', 'trade_loan_kind_code']], values=['account_amt_credit_limit'], index=['application_id'], columns=['trade_loan_kind_code'], aggfunc='mean')

In [28]:
df_test2 = df_test2.fillna(0.0)

In [29]:
df_test2.head()

account_amt_credit_limit                              \
trade_loan_kind_code                     1.0  2.0           3.0      4.0    
application_id                                                              
51534101                        129021.220000  0.0   7885.353571      0.0   
51534210                          5300.000000  0.0   5520.562500      0.0   
51534377                             0.000000  0.0   5666.666667      0.0   
51534467                         59863.290000  0.0      0.000000  15000.0   
51534573                         45269.272727  0.0  30263.625714  16856.0   

                                                       
trade_loan_kind_code     5.0  6.0           7.0  99.0  
application_id                                         
51534101                  0.0  0.0      0.000000  0.0  
51534210                  0.0  0.0      0.000000  0.0  
51534377              28000.0  0.0      0.000000  0.0  
51534467              15000.0  0.0      0.000000  0.0  
51534573              37600.0  0.0  63333.333333  0.0